# Starfinder: Soldier with Magnetar Rifle

Advanced weapons are available to all martial characters through the {srd_feats}`Weapon Proficiency <911-weapon-proficiency>` general feat.
However, with Weapon Proficiency, when you hit level 5 your proficiency with martial weapons increases to expert, but not that with advanced weapons. This is unlike ancestral Weapon Familiarity feats.

The {srd_weapons}`Magnetar Rifle <68-magnetar-rifle>` is an advanced weapon (d12 Analog, Automatic, range 60ft, magazine 30, reload 1).
The closest match with martial weapons are the substantially worse {srd_weapons}`Machine Gun <55-machine-gun>` (d8 Analog, Automatic, range 40ft, magazine 20, reload 2) or {srd_weapons}`Rotolaser <58-rotolaser>` (d8 Automatic, Tech, range 30ft, magazine 10~100 depending on level, reload 1)

Crucially, Area Fire and Auto-Fire use one's class proficiency, not the weapon proficiency - which raises the question of how do the damage profiles for these weapons compare.
So a {srd_classes}`Soldier<5-soldier>` using a Magnetar Rifle would use their lowered weapon proficiency for Primary Target and simple Strikes, and their full class proficiency for Auto-fire. At level 7+, this also lowers the benefit from Weapon Specialization.

Let's analyse a full round of firing a rotolaser and compare it with a magnetar rifle.
With both weapons, we'll do Primary target -> Auto-Fire against a single target -> simple Strike.

In [ ]:
# Install in jupyterlite
%pip install -q pathfinder2e-stats

import xarray

import pathfinder2e_stats as pf2

In [ ]:
martial_atk = pf2.tables.SIMPLE_PC.weapon_attack_bonus.soldier.sum("component")
advanced_atk = (
    martial_atk
    - pf2.tables.PC.weapon_proficiency.soldier
    + pf2.tables.PC.weapon_proficiency.weapon_proficiency
)

atk_by_level = xarray.concat(
    [martial_atk, advanced_atk],
    dim="weapon",
)
atk_by_level.coords["weapon"] = ["rotolaser", "magnetar_rifle"]
atk_by_level.display(transpose=True)

In [ ]:
level = 5
atk = atk_by_level.sel(level=level)
area_fire_DC = (
    pf2.tables.SIMPLE_PC.area_fire_DC.soldier.sum("component").sel(level=level).item()
)
print(f"{area_fire_DC=}")

In [ ]:
weapon_dice = pf2.tables.PC.weapon_dice.improvement.sel(level=level).item()
martial_weapon_specialization = pf2.tables.PC.weapon_specialization.soldier.sel(
    level=level
).item()
rotolaser = pf2.armory.starfinder.ranged.rotolaser(
    weapon_dice, martial_weapon_specialization
)

rotolaser

In [ ]:
# This is bespoke to the combination of class and Weapon Proficiency feat,
# so we need to handwrite it.
advanced_weapons_specialization = (
    xarray.DataArray([0] * 11 + [2] * 9, dims=["level"], coords={"level": range(1, 21)})
    .sel(level=level)
    .item()
)
magnetar_rifle = pf2.armory.starfinder.ranged.magnetar_rifle(
    weapon_dice, advanced_weapons_specialization
)

magnetar_rifle

In [ ]:
enemy = pf2.tables.SIMPLE_NPC.sel(level=level, drop=True)[["AC", "saving_throws", "HP"]]
enemy.display()

In [ ]:
# both 'weapon' and 'challenge' are what-if analyses - let's compare the same dice rolls
# against progressively harder-to-hit enemies.
pf2.set_config(
    check_dependent_dims=("challenge", "weapon"),
    damage_dependent_dims=("challenge",),
)

In [ ]:
primary_target = pf2.check(atk, DC=enemy.AC)
primary_target = xarray.concat(
    [
        pf2.damage(primary_target.sel(weapon="rotolaser"), rotolaser),
        pf2.damage(primary_target.sel(weapon="magnetar_rifle"), magnetar_rifle),
    ],
    dim="weapon",
    join="outer",
    fill_value=0,
)

In [ ]:
auto_fire = pf2.check(
    enemy.saving_throws, DC=area_fire_DC, primary_target=primary_target
)

auto_fire = xarray.concat(
    [
        pf2.damage(
            auto_fire.sel(weapon="rotolaser"),
            rotolaser.area_fire(),
        ),
        pf2.damage(
            auto_fire.sel(weapon="magnetar_rifle"),
            magnetar_rifle.area_fire(),
        ),
    ],
    dim="weapon",
    join="outer",
    fill_value=0,
)

In [ ]:
# Note: Primary Target does not increase MAP, but Auto-Fire does
third_strike = pf2.check(atk - 5, DC=enemy.AC)
third_strike = xarray.concat(
    [
        pf2.damage(third_strike.sel(weapon="rotolaser"), rotolaser),
        pf2.damage(third_strike.sel(weapon="magnetar_rifle"), magnetar_rifle),
    ],
    dim="weapon",
    join="outer",
    fill_value=0,
)

In [ ]:
full_round = xarray.concat([primary_target, auto_fire, third_strike], dim="action")
full_round["action"] = ["primary_target", "auto_fire", "third_strike"]

## Chance to hit
Note how the saving throw against auto-fire uses the same DC for both weapons, but it is influenced by the outcome of the Primary Target strike.

In [ ]:
pf2.outcome_counts(full_round).stack(
    row=["action", "outcome"], col=["challenge", "weapon"]
).to_pandas()

## Mean damage

In [ ]:
total_damage = full_round.total_damage.mean("roll")
total_damage = xarray.concat(
    [total_damage, total_damage.sum("action").expand_dims(action=["TOTAL"])],
    dim="action",
)
total_damage = total_damage.stack(col=["challenge", "weapon"]).to_pandas()
total_damage

## Damage distribution

In [ ]:
bins = full_round.total_damage.max().item() + 1
_ = (
    full_round.total_damage.stack(col=["challenge", "weapon"])
    .sum("action")
    .to_pandas()
    .hist(bins=bins, sharex=True, figsize=(10, 10))
)